In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df2 = pd.read_csv('last12months/12months/220330932.csv')
df2['dt'] = pd.to_datetime(df2['TIMESTAMP'],unit='ms')


/home/leonardo/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21,23,25,27,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def gotAnomalies(df:pd.DataFrame):
    unique = df['Anomalia #1'].unique()
    b = False
    for v in unique:
        if v == 0 or v == np.nan:
            b = False
        else:
            b = True
            break
    return b, unique


In [4]:
print(gotAnomalies(df2))

(True, array([nan,  0., 76.]))


In [5]:
def cleanCSV(df):
  cols_to_check = df.columns
  arr_col = []
  for col in df.columns:
    if df[col].isna().all():
        print("La colonna", col, "contiene solo valori NaN.")
        arr_col.append(col)
  df_cleaned = df.drop(columns=arr_col)
  df_cleaned = df_cleaned.replace('---', 0)
  return df_cleaned

In [6]:
def sortDf(df):
    if 'TIMESTAMP' in df.columns:
        df = df.sort_values('TIMESTAMP',ascending=True)
        df.index = range(len(df))
    else:
        print("La colonna TIMESTAMP non è presente nel dataframe.")
        return None
    return df

In [7]:
def resampleDf(df,datetime):
    if 'dt' in df.columns:
        df.set_index('dt', inplace=True)
    df_noNan = df.ffill()
    df_resampled = df_noNan.resample(datetime).ffill()
    df_resampled = df_resampled.replace('---', 0)
    return df_resampled
    

In [8]:
def checkFileInFolder(folder, file):
    if file in os.listdir(folder):
        return True
    return False

In [10]:
def getResampledDf(df,file_path):
    if not checkFileInFolder(folder ='last12months/',file=file_path):
        df_sorted = sortDf(df)
        df_sorted = cleanCSV(df_sorted)
        df_resampled = resampleDf(df_sorted,'31min')
        df_resampled.to_csv('last12months/' + file_path)
        return df_sorted, df_resampled
    else:
        df_resampled = pd.read_csv('last12months/'+ file_path)
        return None, df_resampled

    

In [11]:
df2_sorted, df2_resampled = getResampledDf(df2,'220330932_resampled30s.csv')

La colonna Modo Impianto contiene solo valori NaN.


In [12]:
df2_resampled['Anomalia #1'] = df2_resampled['Anomalia #1'].replace(np.nan,0)
df2_resampled.head()

,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,Energia totale globale,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
dt,,,,,,,,,,,,,,,,,,,,,
2022-06-30 21:42:00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30 22:13:00,1.656626e+12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30 22:44:00,1.656626e+12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30 23:15:00,1.656626e+12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-30 23:46:00,1.656626e+12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def getUniques(df):
    cols = df.columns
    dict_unique = dict.fromkeys(cols,None)
    for col in cols:
        dict_unique[col] = df[col].unique()
    return dict_unique

In [14]:
def getBooelan(dict):
    arr = []
    for key, value in dict.items():
        if len(value) < 4:
            b = True
            for v in value:
                # print(v,pd.isna(v))
                if pd.isna(v)==False:
                    if int(v) != 0 and int(v) != 1:
                        # print('ciao')
                        b = False
                        break
            # print(value,b)
            if b:
                arr.append(key)
    return arr

In [15]:
def printUniques(dict,tp):
    booleans = getBooelan(dict)
    for key, value in dict.items():
        counts = []
        if key in booleans and tp ==1:
            print(key, value)
        elif tp==2 and len(value) > 4 :
            print(key, value[0:5])

In [16]:
def countOccurences(df,dict):
    booleans = getBooelan(dict)
    counters = dict.fromkeys(booleans,None)
    for key, value in dict.items():
        if key in booleans:
            value = value[1:]
            counts = dict.fromkeys(value,None)
            for v in value:
                if not pd.isna(v):
                    counts[v]= df[key].value_counts()[v]
            counters[key] = counts
            # print(key, counts, max(counts))
    return counters
            

In [17]:
def getMax(dict):
    maxs = dict.fromkeys(dict.keys(),None)
    for key, value in dict.items():
        # print(key, value)
        for k, v in value.items():
            if v == max(value.values()):
                maxs[key] = k
    return maxs

In [18]:
def plotColumn(df,col):
    if col not in df.columns:
        print("La colonna", col, "non è presente nel dataframe.")
        return None
    df_toPlot = df[col]
    plt.plot(df_toPlot)
    try: 
        plt.ylim(0, max(df_toPlot.unique())+1)
    except:
        pass

    plt.show()


In [19]:
dict_unique2 = getUniques(df2_resampled)
arr_booleani2 = getBooelan(dict_unique2)


In [20]:
def getLastValues(df):
    d_sample = df.iloc[-1]
    dictionary = dict.fromkeys(df.columns,None)

    for col in df.columns:
        if col in d_sample.index:
            dictionary[col] = d_sample[col]

    return dictionary

In [21]:
def reversedMax(dizionario):
    for key, value in dizionario.items():
        if value == 1:
            dizionario[key] = 0
        else:
            dizionario[key] = 1
    return dizionario

In [22]:
def getGeneralBooleans(df2):
    b2 = getBooelan(getUniques(df2))
    intersection = list(set(b2))
    intersection.sort()
    return intersection

In [23]:
def fillBooleans(df,booleans,dizionario):
    for key in booleans:
        if key != 'Modalità Estate/Inverno (solo scrittura)' and key in dizionario.keys():
            print(f'{key}: {dizionario[key]}')        
            df[key] = df[key].replace(np.nan,dizionario[key])
    return df

In [24]:
def fillOthers(df,sample):
    col_s = sample.index
    booleani = getBooelan(getUniques(df))
    for col in df.columns:
        # print(col)
        if col == 'Anomalia #1':
            df[col] = df[col].replace(np.nan,sample[col])
        if col not in booleani and col in col_s:
            # print('in')
            df[col] = df[col].replace(np.nan,sample[col])
    return df

In [25]:
print("Anomalies' values: ",df2_resampled['Anomalia #1'].unique())
num_anomaliesDf2 = df2['Anomalia #1'].value_counts()[76]
print("Number of anomalies type 76 in df2:", num_anomaliesDf2)

Anomalies' values:  [ 0. 76.]
Number of anomalies type 76 in df2: 9


In [26]:
num_anomaliesDf2 = df2_resampled['Anomalia #1'].value_counts()[76]
print("Number of anomalies type 76 in df2:", num_anomaliesDf2)

Number of anomalies type 76 in df2: 28


In [27]:
column_to_first_valid_index = {}

for column in df2_resampled.columns:
  first_valid_index = df2_resampled[column].first_valid_index()
  column_to_first_valid_index[column] = first_valid_index

#iterate the dict and print the key and value
for key, value in column_to_first_valid_index.items():
  print(key, ' : ', value)

TIMESTAMP  :  2022-06-30 22:13:00
Abilitazione al conteggio energia  :  2022-07-22 09:20:00
Anomalia #1  :  2022-06-30 21:42:00
Caldaia  :  2022-07-17 22:23:00
Caldaia giornaliero  :  2022-07-21 22:29:00
Cloud Status  :  2022-07-22 09:20:00
Connection Status  :  2022-07-22 09:20:00
Energia parziale in riscaldamento caldaia  :  2022-07-22 09:51:00
Energia parziale in sanitario caldaia  :  2022-07-22 11:24:00
Energia totale globale  :  2022-07-22 09:51:00
Energia totale globale delta  :  2022-07-22 11:24:00
Energia totale globale per day  :  2022-07-21 22:29:00
Energia totale globale per month  :  2022-06-30 22:13:00
Energia totale globale per week  :  2022-07-17 22:23:00
Energia totale in riscaldamento caldaia  :  2022-07-22 09:51:00
Energia totale in sanitario caldaia  :  2022-07-22 11:24:00
Fattore di correzione  :  2022-07-22 09:20:00
Flame status  :  2022-07-22 09:20:00
Flusso sanitario  :  2022-07-22 09:20:00
Intercetta della retta della potenza  :  2022-07-22 09:20:00
Livello modu

In [ ]:
dfProva = df2_resampled.copy()
dfProva = dfProva.dropna()

In [ ]:
dfProva.head()

In [ ]:
dfProva.shape

## Work with final DataFrame

In [28]:
df = pd.read_csv('last12months/12months/220330932_resampled30s_noNan.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'last12months/12months/220330932_resampled30s_noNan.csv'

In [ ]:
df.head()

In [ ]:
dfAnomalies = df[df['Anomalia #1'] == 76]

In [ ]:
dfAnomalies

In [ ]:
datesAnomalies = dfAnomalies['dt']
#Create a new column month that contains only the month of the date
datesAnomalies = pd.to_datetime(datesAnomalies)
datesAnomalies = datesAnomalies.dt.month


In [ ]:
months = datesAnomalies.unique()

In [ ]:
months

## Data Mining

In [ ]:
df2 = pd.read_csv('last12months/12months/220330932.csv')



In [ ]:
ts = df2['TIMESTAMP']
delta_t = np.diff(ts)

media = np.mean(delta_t)



In [ ]:
media/1000

In [ ]:
seconds=(media/1000)/3600/24
seconds

In [ ]:
df2['Anomalia #1'].unique()

In [ ]:
final = pd.read_csv('last12months/12months/220330932_resampled30s_noNan.csv')

In [ ]:
print(gotAnomalies(final))

In [ ]:
dfWithAnomalies = final[final['Anomalia #1'] != 0]

In [ ]:
dfWithAnomalies.head()

In [ ]:
final.head()

In [ ]:
final['Normal/Attack'] = final['Anomalia #1'].apply(lambda x: 'Attack' if x != 0 else 'Normal')

In [ ]:
final['Normal/Attack'].unique()

In [ ]:
dfA = final.iloc[0:55820]
df = final.iloc[55820:]
dfN = dfA.loc[dfA['Normal/Attack'] == 'Normal'].reset_index(drop=True)


In [ ]:
df.to_csv('last12months/baxi/BAXI_220330932_Normal.csv')
dfA.to_csv('last12months/baxi/BAXI_220330932_Attack.csv')

In [ ]:
df['TIMESTAMP'] = df['TIMESTAMP'].str.strip()
dfA['TIMESTAMP'] = dfA['TIMESTAMP'].str.strip()

In [ ]:
swat = pd.read_csv()

## Create datasets for the notebook

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [4]:
df = pd.read_csv('last12months/12months/220330932_resampled60min_noNan.csv')
dfWithAnomalies = df[df['Anomalia #1'] != 0]
df['Normal/Attack'] = df['Anomalia #1'].apply(lambda x: 'Attack' if x != 0 else 'Normal')

In [5]:
df['Normal/Attack'].unique()

array(['Normal', 'Attack'], dtype=object)

In [6]:
dfWithAnomalies.head()

,dt,TIMESTAMP,Abilitazione al conteggio energia,Anomalia #1,Caldaia,Caldaia giornaliero,Cloud Status,Connection Status,Energia parziale in riscaldamento caldaia,Energia parziale in sanitario caldaia,...,Stato attivazione BOOST sanitario,Taratura sensore,Temperatura di mandata,Temperatura esterna,Temperatura esterna mediata,Temperatura fumi,Temperatura sanitario,Temperatura scambiatore,Velocita' ventilatore,WiFi signal
462,2023-02-02 03:00:00,1675306785648,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,20.3,0.0,0.0,0.0,18.3,0.0,0.0,-50.0
463,2023-02-02 04:00:00,1675310393588,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,19.1,0.0,0.0,0.0,17.3,0.0,0.0,-49.0
464,2023-02-02 05:00:00,1675313993686,1.0,76.0,70625003.0,10330337.0,2.0,1.0,238.0,34.0,...,0.0,-2.5,18.1,0.0,0.0,0.0,16.6,0.0,0.0,-50.0


In [12]:
dfA = df.iloc[0:2000]
df = df.iloc[2000:]
dfN = dfA.loc[dfA['Normal/Attack'] == 'Normal'].reset_index(drop=True)

In [14]:
df.to_csv('data/baxi/BAXI_220330932_Normal.csv')
dfA.to_csv('data/baxi/BAXI_220330932_Attack.csv')